In [23]:
import tenseal.sealapi as seal
from polynomial import enc_poly
import util
from numpy.polynomial.polynomial import polymul

In [24]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
poly_modulus_degree = 2**14
mod_pow = 35
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, [60,mod_pow,mod_pow, mod_pow,mod_pow,mod_pow,mod_pow,mod_pow,mod_pow,60]))
scale = pow(2.0, mod_pow)

# What are the specific primes in the modulus chain? These help calculate exact scales 
# of rescaled ciphertexts later on in this notebook. 
primes = [modulus.value() for modulus in parms.coeff_modulus()]

context = seal.SEALContext(parms,True,seal.SEC_LEVEL_TYPE.TC128)

util.print_parameters(context)

print("|\t Max Bit Count: " + str(seal.CoeffModulus.MaxBitCount(poly_modulus_degree, seal.SEC_LEVEL_TYPE.TC128)))

/
|Encryption parameters: 
|	scheme: CKKS
|	poly_modulus_degree: 16384
|	coeff_modulus_size: 400 (60 35 35 35 35 35 35 35 35 60) bits
|	 Max Bit Count: 438


All keys geneated using `seal::KeyGenerator`. We also need an `encoder`, `evaluator`, `encryptor`, and `decryptor`. 

In [25]:

keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()

public_key = seal.PublicKey()
keygen.create_public_key(public_key)

relin_keys = seal.RelinKeys()
keygen.create_relin_keys(relin_keys)

galois_keys = seal.GaloisKeys()
keygen.create_galois_keys()

encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

encoder = seal.CKKSEncoder(context)
slot_count = seal.CKKSEncoder.slot_count(encoder)

print('Number of slots: ' + str(slot_count))

Number of slots: 8192


Let $x$ be the encrypted difference between two number in $a,b\in[0,1]$, such that $x\in[-1,1]$.

In [26]:
a=0.2;b=0.8

x = a-b
x_plain = seal.Plaintext()
encoder.encode(x,scale, x_plain)

x_enc = seal.Ciphertext()
encryptor.encrypt(x_plain, x_enc)

The polynomial we want to calculate is
* $g(x) = (35x-35x^3+21x^5-5x^7)/2^4$

which is done using the function `enc_poly` from the `polynomial` module. 

In [27]:
coeffs = [35, -35, 21, -5]
degrees = [1, 3, 5, 7]

power = 4

enc_g_x = enc_poly(x_enc, coeffs, degrees, power, evaluator, context, encoder,  parms, relin_keys, encryptor)

Note that the resulting ciphertext has a different scale, $\tilde{\Delta}$, compared to the bit-sizes of the modulus chain. This means we can not achieve scale stabilization. 
To compensate for this, manually change the scale of the resulting CT 
such that it matches the original scale, $\Delta$. 

- Note that `enc_g_x` is an encryption of $\tilde{\Delta}g(x)$ with scale $\tilde{\Delta}$
- Multiplying with plaintext $\Delta/\tilde{\Delta}$ which has scale $\tilde{\Delta}$ will yield an encryption of $\Delta g(x)$ with scale $\tilde{\Delta}^2$

Multiplying by the plain factor $\Delta/\tilde{\Delta}$ and manually changing the scale gives an encryption of $\Delta g(x)$ with the correct scale factor.

In [28]:
util.print_info(enc_g_x, decryptor, context, encoder, (35*x-35*x**3+21*x**5-5*x**7)/2**4)

# Code below introduces additional error. Question is if 
# this added error in practice eliminates the benefit
# of using the composite polynomial. 

scaling_factor = scale/enc_g_x.scale/2**power

plain_factor = seal.Plaintext()
encoder.encode(scaling_factor, enc_g_x.scale, plain_factor)
evaluator.mod_switch_to_inplace(plain_factor, enc_g_x.parms_id())

evaluator.multiply_plain_inplace(enc_g_x, plain_factor)
evaluator.rescale_to_next_inplace(enc_g_x)
enc_g_x.scale = scale

util.print_info(enc_g_x, decryptor, context, encoder, (35*x-35*x**3+21*x**5-5*x**7)/2**4)

	Enc. result:	-0.9330960635156593
	Plain result:	-0.933312
	Scale:	549887427425.6552
	Ch. ind:	4
	Enc. result:	-0.9333817853662447
	Plain result:	-0.933312
	Scale:	34359738368.0
	Ch. ind:	3


Now we are ready to calculate $f(g(x))$ where 
* $f(x) = (4589x-16577x^3+25614x^5-12860x^7)/2^{10}$

In [29]:
# Not working, unsure why scale is out of bound. With this method,
# we should achieve scale stabilization. Check scale stabilization 
# in square and multiply. 
coeffs = [4589, -16577, 25614, -12860]
power = 10


enc_compare = enc_poly(enc_g_x, coeffs, degrees, power, evaluator, context, encoder, parms, relin_keys, encryptor)

ValueError: scale out of bounds

In [ ]:
util.print_info(enc_compare, decryptor, context, encoder, (35*x-35*x**3+21*x**5-5*x**7)/2**4)

	Enc. result:	-0.9330961981934047
	Plain result:	-0.933312
	Scale:	549887427425.6552
	Ch. ind:	3
